# Project 1 - Drug annotation of 23andme report

## Team 14  

- Meagan Burt ()

- Lingrui Cai 

- Kyle Contjes ()

- Catherine Kenney ()



## Part I. Answer to the questions

### Meagan Burt
1.What was your biggest challenge in this project? (regarding writing code and not only)

2.What did you learn while working on this project? (regarding writing code and not only)

3.If you had more time on the project what other question(s) would you like to answer? (at least one question is required)

### Lingrui Cai
1.What was your biggest challenge in this project? (regarding writing code and not only)

2.What did you learn while working on this project? (regarding writing code and not only)

3.If you had more time on the project what other question(s) would you like to answer? (at least one question is required)

### Kyle Contjes
1.What was your biggest challenge in this project? (regarding writing code and not only)

2.What did you learn while working on this project? (regarding writing code and not only)

3.If you had more time on the project what other question(s) would you like to answer? (at least one question is required)

### Catherine Kenney
1.What was your biggest challenge in this project? (regarding writing code and not only)

2.What did you learn while working on this project? (regarding writing code and not only)

3.If you had more time on the project what other question(s) would you like to answer? (at least one question is required)

## Part II Solution

####  Read in files and data preparation

23andme_v5_hg19_ref.txt.gz download from https://github.com/arrogantrobot/23andme2vcf

PharmGKB download from https://www.pharmgkb.org/downloads

Annotation data -> Variant and Clinical Annotations Data -> variantAnnotations.zip

https://api.pharmgkb.org/v1/download/file/data/variantAnnotations.zip

#### 1. Map/merge the 23andme file and the variant-drug annotation file based on dbSNP_ID (also known as rsID).

In [3]:
import pandas as pd
import numpy as np

In [5]:
var_drugs = pd.read_csv('var_drug_ann.tsv', sep = '\t', error_bad_lines = False)
inds = var_drugs.index
var_drugs.index = inds + 1
var_drugs.head()

b'Skipping line 4442: expected 11 fields, saw 13\n'


,Variant Annotation ID,Variant/Haplotypes,Gene,Drug(s),PMID,Phenotype Category,Significance,Notes,Sentence,Alleles,Specialty Population
1,1451159680,rs5031016,CYP2A6,warfarin,22248286,Dosage,no,No association was found between this variant ...,Allele G is not associated with increased dose...,G,NaN
2,1183684657,CYP2D6 ultrarapid metabolizer genotype,CYP2D6,tramadol,18204346,Metabolism/PK,yes,"Median (+)R,R-tramadol area under the curve wa...",CYP2D6 ultra-metabolizer genotype is associate...,NaN,NaN
3,1451306860,CYP2C9*11,CYP2C9,warfarin,33350885,Dosage,not stated,This case suggests that CYP2C9 *11/*11 carrier...,CYP2C9 *11/*11 is associated with decreased do...,*11/*11,NaN
4,1448997750,"CYP2B6*1, CYP2B6*18",CYP2B6,efavirenz,16495778,Metabolism/PK,yes,Please note that in the paper the allele was r...,CYP2B6 *1/*18 is associated with increased con...,*1/*18,NaN
5,1448631821,"CYP2C19*1, CYP2C19*2",CYP2C19,"clomipramine,""desmethyl clomipramine""",28470111,Metabolism/PK,no,in a single individual,CYP2C19 *1/*2 is associated with increased tro...,*1/*2,NaN


In [6]:
ref_file_head = ["CHR", "POS", "dbSNP_ID", "ALLELE"]
ref_file = pd.read_csv('23andme_v5_hg19_ref.txt', sep = '\t')
ref_file.columns = ref_file_head
ref_file.head()

,CHR,POS,dbSNP_ID,ALLELE
0,chr1,74792,rs13328684,G
1,chr1,565508,rs9283150,G
2,chr1,726912,i713426,A
3,chr1,727841,rs116587930,G
4,chr1,752721,rs3131972,A


In [7]:
filtered_vd = var_drugs[var_drugs['Variant/Haplotypes'].str.startswith('rs')]
filtered_vd = filtered_vd.rename(columns = {"Variant/Haplotypes": "dbSNP_ID"})
var_info = pd.merge(filtered_vd, ref_file)
var_info.head()
#var_info is new merged table

,Variant Annotation ID,dbSNP_ID,Gene,Drug(s),PMID,Phenotype Category,Significance,Notes,Sentence,Alleles,Specialty Population,CHR,POS,ALLELE
0,1451159680,rs5031016,CYP2A6,warfarin,22248286,Dosage,no,No association was found between this variant ...,Allele G is not associated with increased dose...,G,NaN,chr19,41349774,A
1,769278059,rs5031016,CYP2A6,nicotine,11779172,Metabolism/PK,not stated,This SNP is in the CYP2A6*10 and *7 alleles ex...,Allele G is associated with decreased metaboli...,G,NaN,chr19,41349774,A
2,769278041,rs5031016,CYP2A6,nicotine,12445030,"Other,""Metabolism/PK""",no,This is a SNP within the CYP2A6*7 or *10 allel...,Allele G is associated with decreased metaboli...,G,NaN,chr19,41349774,A
3,1451307920,rs7624046,KCNMB2,ritodrine,32371615,Efficacy,yes,variant-type homozygote carriers of rs7624046 ...,Genotype TT is associated with decreased respo...,TT,NaN,chr3,178461771,C
4,1449716734,rs1128503,ABCB1,fentanyl,29601950,Dosage,yes,Please note that alleles have been complemente...,Genotype AA is associated with increased dose ...,AA,NaN,chr7,87179601,A


#### 2. Filter the output so that it only contains significant associations (SIGNIFICANCE is yes) for variants that affect the drug efficacy (`PHENOTYPE_CATEGORY` is `efficacy`).

In [16]:
var_info = var_info[(var_info.Significance == "yes") & (var_info['Phenotype Category'] == "Efficacy")]
var_info

,Variant Annotation ID,dbSNP_ID,Gene,Drug(s),PMID,Phenotype Category,Significance,Notes,Sentence,Alleles,Specialty Population,CHR,POS,ALLELE
3,1451307920,rs7624046,KCNMB2,ritodrine,32371615,Efficacy,yes,variant-type homozygote carriers of rs7624046 ...,Genotype TT is associated with decreased respo...,TT,NaN,chr3,178461771,C
8,1449716295,rs1128503,ABCB1,fentanyl,22277678,Efficacy,yes,Please note that alleles have been complemente...,Genotype AA is associated with decreased respo...,AA,NaN,chr7,87179601,A
9,1446897429,rs1128503,ABCB1,glucocorticoids,25346426,Efficacy,yes,The GG genotype was more frequent in patients ...,Genotype GG is associated with decreased respo...,GG,NaN,chr7,87179601,A
32,981238152,rs1128503,ABCB1,antipsychotics,22909202,Efficacy,yes,Please note: this variant deviated from Hardy-...,Allele G is associated with increased response...,G,NaN,chr7,87179601,A
35,1449748447,rs1128503,ABCB1,tacrolimus,29615122,Efficacy,yes,Patients with the AA genotype had a higher eff...,Genotype AA is associated with increased respo...,AA,NaN,chr7,87179601,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5265,769169630,rs4796793,STAT3,interferons,17602083,Efficacy,yes,IFN-alpha. Careful- GC SNP. Based on dbSNP pop...,Genotype GG is associated with increased respo...,GG,NaN,chr17,40542210,G
5272,1448995971,rs2233945,PSORS1C1,etanercept,28107378,Efficacy,yes,The degree of response was determined by EULAR...,Allele C is associated with decreased response...,C,NaN,chr6,31107361,C
5274,1184997439,rs2900478,SLCO1B1,hmg coa reductase inhibitors,25350695,Efficacy,yes,Carriers of this variant respond to statins wi...,Allele A is associated with decreased response...,A,NaN,chr12,21368797,T
5279,1446902880,rs4742610,PTPRD,"trandolapril,""verapamil""",26425837,Efficacy,yes,This was associated with resistant hypertensio...,Allele T is associated with resistance to tran...,T,NaN,chr9,9687487,C


#### 3. Save the output of the filtering step in a tab-separated file (23andme_PharmGKB_map.tsv) with the following columns:

#### 4. Create a tab separated file with summarized data with the following columns:

#### 5. Plot the distribution of the number of drugs associated with a gene, and the number of SNPs for a gene. 